# Ensembles EDA
In this notebook, we explore aggregated results from the ensemble models and compare those results to our control and target models.

We also look at the composition of our best ensembles.

## Imports & Settings

In [1]:
# Update working directory to parent so that we may use our custom functions
import os
os.chdir('..')
# os.getcwd( )

import pandas as pd
import matplotlib.pyplot as plt

## Load & Parse Model Results Dataset

Here we load the individual control and target model results. We also parse those results into four additional dataframes:
1. Best individual model epochs by F1 (without control model data)
2. Best individual model epochs by Accuracy (without control model data)
3. Best individual control model epochs by F1 (without target model data)
4. Best individual control model epochs by Accuracy (without target model data)

In [2]:
model_results_df = pd.read_csv('05_results/results_target.csv', converters={'predictions': pd.eval})
model_results_df.head()

,model_name,model_epoch,test_accuracy,test_f1,predictions
0,control_iSarcasm_01,E01_A0.75_F0.41,0.835714,0.454976,"[0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,control_iSarcasm_01,E02_A0.77_F0.39,0.825714,0.452915,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ..."
2,control_iSarcasm_01,E03_A0.75_F0.45,0.784286,0.430189,"[0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ..."
3,control_iSarcasm_01,E04_A0.76_F0.5,0.757857,0.402116,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ..."
4,control_iSarcasm_01,E05_A0.74_F0.45,0.768571,0.408759,"[0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ..."


In [3]:
# get row index for each epoch with best f1 score for each model_name
best_f1_idx = model_results_df.groupby(['model_name'])['test_f1'].transform(max) == model_results_df['test_f1']
# get row index for each epoch with best accuracy score for each model_name
best_acc_idx = model_results_df.groupby(['model_name'])['test_accuracy'].transform(max) == model_results_df['test_accuracy']

# get dataframe filtered by best f1 for each model epoch
best_f1_df = model_results_df[best_f1_idx]
# get dataframe filtered by best accuracy for each model epoch
best_acc_df = model_results_df[best_acc_idx]

In [4]:
# create control model filters
pattern = ".*control.*"
f1_control_filter = best_f1_df['model_name'].str.contains(pattern)
acc_control_filter = best_acc_df['model_name'].str.contains(pattern)

# remove control models to get dataframes with best individual target model 
# epochs by respective metric
f1_estimators_df = best_f1_df[~f1_control_filter]
acc_estimators_df = best_acc_df[~acc_control_filter]

# remove target models to get dataframes with best control model epochs
f1_control_df = best_f1_df[f1_control_filter]
acc_control_df = best_acc_df[acc_control_filter]

## Load & Parse Ensemble Results Dataset

Here we load the ensemble model results. We also parse those results into dataframes according to ensemble type, first by three and five-model ensembles and also by the metric from which those ensembles were created. Recall, one set of ensembles were created from inducers' best epoch selected by best F1 scores and the other set of ensembles were created from inducers' best epoch selected by best Accuracy scores.

In [5]:
# load ensemble results
ensemble_results_df = pd.read_csv('05_results/results_ensemble.csv', converters={'inducers': pd.eval})

In [6]:
ensemble_results_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   ensemble_id    52 non-null     object 
 1   inducers       52 non-null     object 
 2   inducers_spec  52 non-null     object 
 3   dataset        52 non-null     object 
 4   test_accuracy  52 non-null     float64
 5   test_f1        52 non-null     float64
 6   ensemble_type  52 non-null     object 
dtypes: float64(2), object(5)
memory usage: 3.0+ KB


In [7]:
# parse ensemble results by type

# best F1 three and five-model ensembles
f1_TM_ensemble_results_df = ensemble_results_df[ensemble_results_df['ensemble_type'] == 'best-F1_three-model']
f1_FM_ensemble_results_df = ensemble_results_df[ensemble_results_df['ensemble_type'] == 'best-F1_five-model']

# best accuracy three and five-model ensembles
acc_TM_ensemble_results_df = ensemble_results_df[ensemble_results_df['ensemble_type'] == 'best-acc_three-model']
acc_FM_ensemble_results_df = ensemble_results_df[ensemble_results_df['ensemble_type'] == 'best-acc_five-model']

## Plot

In [8]:
# define some plot parameters so that we may loop
f1_plot_dict = {'Control Models': [f1_control_df, 'red', '*', '-.', 'Control Models Mean Scores'],
                'Target Models': [f1_estimators_df, 'blue', 'o', 'dotted', 'Target Models Mean Scores'],
                '3-Inducer Ensembles': [f1_TM_ensemble_results_df, 'orange', 'x', 'solid', '3-IE Mean Scores'], 
                '5-Inducer Ensembles': [f1_FM_ensemble_results_df, 'green', 's', 'dashed', '5-IE Mean Scores']}

acc_plot_dict = {'Control Models': [acc_control_df, 'red', '*', '-.', 'Control Models Mean Scores'],
                 'Target Models': [acc_estimators_df, 'blue', 'o', 'dotted', 'Target Models Mean Scores'],
                 '3-Inducer Ensembles': [acc_TM_ensemble_results_df, 'orange', 'x', 'solid', '3-IE Mean Scores'], 
                 '5-Inducer Ensembles': [acc_FM_ensemble_results_df, 'green', 's', 'dashed', '5-IE Mean Scores']}

In [9]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10.5, 3.5), sharex=True, sharey=False, dpi=1200)

font = {'family':'serif'}

# F1 Plots
ax1.set_title("(a) Best Models Selected via Test F1", fontdict = font)
for k, v in f1_plot_dict.items():
    v[0].plot.scatter(x='test_accuracy', y='test_f1', marker=v[2], c=v[1], alpha=.8, label=k, ax=ax1)
    ax1.axhline(y =v[0]['test_f1'].mean(), color=v[1], linestyle=v[3], lw=1, alpha=.5, linewidth=.7, label=v[4])
    ax1.axvline(x =v[0]['test_accuracy'].mean(), color=v[1], linestyle=v[3], lw=1, alpha=.5, linewidth=.7)

ax1.set_ylim(0.3, 0.55)
ax1.set_xlim(0.72, 0.90)
ax1.legend('', frameon=False)

# Accuracy Plots
ax2.set_title("(b) Best Models Selected via Test Accuracy", fontdict = font)
for k, v in acc_plot_dict.items():
    v[0].plot.scatter(x='test_accuracy', y='test_f1', marker=v[2], c=v[1], alpha=.8, label=k, ax=ax2)
    ax2.axhline(y =v[0]['test_f1'].mean(), color=v[1], linestyle=v[3], lw=1, alpha=.5, linewidth=.7, label=v[4])
    ax2.axvline(x =v[0]['test_accuracy'].mean(), color=v[1], linestyle=v[3], lw=1, alpha=.5, linewidth=.7)

ax2.set_ylim(0.3, 0.55)
ax2.set_xlim(0.72, 0.90)
ax2.legend('', frameon=False)

# get handles and labels
handles, labels = plt.gca().get_legend_handles_labels()

# specify order of items in legend
order = [0,1,2,3,4,5,6,7]

# get the legend object
legends = fig.legend([handles[idx] for idx in order],[labels[idx] for idx in order], 
                     loc='lower center', bbox_to_anchor=(.510, -.15), 
                     fontsize=10, ncol=4)
leg = legends

for line in leg.get_lines():
    line.set_linewidth(2.0)
    
fig.tight_layout()
plt.savefig("figures/scatter_best_epochs_f1_acc.svg", bbox_inches='tight')
plt.show()

## Means & Deltas
The below means are those represented by the h and v lines in the above plot.

In [10]:
print("Plot (a): Best Models Selected via Test F1: F1 Means & Deltas (y-axis)")
for k, v in f1_plot_dict.items():
    mean = v[0]['test_f1'].mean()
    delta = mean - f1_control_df['test_f1'].mean()
    print(f"\t{k} Mean F1: {round(mean, 3)} delta: {round(delta, 3)}")

print("\n Plot (a): Best Models Selected via Test F1: Accuracy Means & Deltas (x-axis)")
for k, v in f1_plot_dict.items():
    mean = v[0]['test_accuracy'].mean()
    delta = mean - f1_control_df['test_accuracy'].mean()
    print(f"\t{k} Mean Accuracy: {round(mean, 3)} delta: {round(delta,3)}")

Plot (a): Best Models Selected via Test F1: F1 Means & Deltas (y-axis)
	Control Models Mean F1: 0.444 delta: 0.0
	Target Models Mean F1: 0.423 delta: -0.021
	3-Inducer Ensembles Mean F1: 0.485 delta: 0.041
	5-Inducer Ensembles Mean F1: 0.499 delta: 0.055

 Plot (a): Best Models Selected via Test F1: Accuracy Means & Deltas (x-axis)
	Control Models Mean Accuracy: 0.843 delta: 0.0
	Target Models Mean Accuracy: 0.811 delta: -0.032
	3-Inducer Ensembles Mean Accuracy: 0.854 delta: 0.011
	5-Inducer Ensembles Mean Accuracy: 0.862 delta: 0.02


In [11]:
print("Plot (b): Best Models Selected via Test Accuracy: F1 Means & Deltas (y-axis)")
for k, v in acc_plot_dict.items():
    mean = v[0]['test_f1'].mean()
    delta = mean - acc_control_df['test_f1'].mean()
    print(f"\t{k} Mean F1: {round(mean, 3)} delta: {round(delta,3)}")

print("\n Plot (b): Best Models Selected via Test Accuracy: Accuracy Means & Deltas (x-axis)")
for k, v in acc_plot_dict.items():
    mean = v[0]['test_accuracy'].mean()
    delta = mean - acc_control_df['test_accuracy'].mean()
    print(f"\t{k} Mean Accuracy: {round(mean, 3)} delta: {round(delta,3)}")

Plot (b): Best Models Selected via Test Accuracy: F1 Means & Deltas (y-axis)
	Control Models Mean F1: 0.431 delta: 0.0
	Target Models Mean F1: 0.396 delta: -0.035
	3-Inducer Ensembles Mean F1: 0.449 delta: 0.018
	5-Inducer Ensembles Mean F1: 0.448 delta: 0.017

 Plot (b): Best Models Selected via Test Accuracy: Accuracy Means & Deltas (x-axis)
	Control Models Mean Accuracy: 0.845 delta: 0.0
	Target Models Mean Accuracy: 0.833 delta: -0.012
	3-Inducer Ensembles Mean Accuracy: 0.87 delta: 0.026
	5-Inducer Ensembles Mean Accuracy: 0.875 delta: 0.03


## Inducer Explorations
Below, we explore the composition of the best inducers and how they compare to the control models. We explore questions such as "What percentage of our ensembles outperform the best control model performance on each metric?" and "What inducers compose the top 10 ensembles?"

In [12]:
# get all control model epochs
filter = model_results_df['model_name'].str.contains(pattern)

control_results = model_results_df[filter]
control_results.head()

,model_name,model_epoch,test_accuracy,test_f1,predictions
0,control_iSarcasm_01,E01_A0.75_F0.41,0.835714,0.454976,"[0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,control_iSarcasm_01,E02_A0.77_F0.39,0.825714,0.452915,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ..."
2,control_iSarcasm_01,E03_A0.75_F0.45,0.784286,0.430189,"[0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ..."
3,control_iSarcasm_01,E04_A0.76_F0.5,0.757857,0.402116,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ..."
4,control_iSarcasm_01,E05_A0.74_F0.45,0.768571,0.408759,"[0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ..."


In [13]:
# view mins and maxes
control_results.describe()

,test_accuracy,test_f1
count,30.000000,30.000000
mean,0.797881,0.410463
std,0.043815,0.023087
min,0.672857,0.367403
25%,0.772321,0.394061
50%,0.812500,0.410196
75%,0.827857,0.429512
max,0.850000,0.454976


### Ensembles Created Based on Inducer Best Epoch by F1 

In [14]:
# get all ensembles created via inducer test_F1 scores
best_f1_ensembles = pd.concat([f1_TM_ensemble_results_df, f1_FM_ensemble_results_df])
len(best_f1_ensembles)


26

In [15]:
# find how many ensembles have a better test_F1 score than the best control model
better_f1_count = best_f1_ensembles['test_f1'][best_f1_ensembles['test_f1'] > control_results['test_f1'].max()].count()
better_f1_count

23

In [16]:
# find how many ensembles have a better test_accuracy score than the best control model
better_acc_count = best_f1_ensembles['test_accuracy'][best_f1_ensembles['test_accuracy'] > control_results['test_accuracy'].max()].count()
better_acc_count

19

In [17]:
# get top 10 ensembles by test_F1 score
top_10_f1 = best_f1_ensembles.nlargest(10, 'test_f1')
top_10_f1

,ensemble_id,inducers,inducers_spec,dataset,test_accuracy,test_f1,ensemble_type
20,ensemble_5,"[target-iSarcasm_inter-HellaSwag, target-iSarc...","('target-iSarcasm_inter-HellaSwag_01', 'target...",iSarcasm_test,0.869286,0.522193,best-F1_five-model
0,ensemble_17,"[target-iSarcasm_inter-SARC, target-iSarcasm_i...","('target-iSarcasm_inter-SARC_01', 'target-iSar...",iSarcasm_test,0.863571,0.516456,best-F1_three-model
1,ensemble_18,"[target-iSarcasm_inter-SARC, target-iSarcasm_i...","('target-iSarcasm_inter-SARC_01', 'target-iSar...",iSarcasm_test,0.857857,0.511057,best-F1_three-model
2,ensemble_16,"[target-iSarcasm_inter-SARC, target-iSarcasm_i...","('target-iSarcasm_inter-SARC_01', 'target-iSar...",iSarcasm_test,0.848571,0.506977,best-F1_three-model
21,ensemble_1,"[target-iSarcasm_inter-IMDB, target-iSarcasm_i...","('target-iSarcasm_inter-IMDB_01', 'target-iSar...",iSarcasm_test,0.869286,0.504065,best-F1_five-model
22,ensemble_4,"[target-iSarcasm_inter-IMDB, target-iSarcasm_i...","('target-iSarcasm_inter-IMDB_01', 'target-iSar...",iSarcasm_test,0.855000,0.503667,best-F1_five-model
3,ensemble_10,"[target-iSarcasm_inter-HellaSwag, target-iSarc...","('target-iSarcasm_inter-HellaSwag_01', 'target...",iSarcasm_test,0.861429,0.502564,best-F1_three-model
4,ensemble_11,"[target-iSarcasm_inter-HellaSwag, target-iSarc...","('target-iSarcasm_inter-HellaSwag_01', 'target...",iSarcasm_test,0.855714,0.502463,best-F1_three-model
5,ensemble_12,"[target-iSarcasm_inter-HellaSwag, target-iSarc...","('target-iSarcasm_inter-HellaSwag_01', 'target...",iSarcasm_test,0.869286,0.498630,best-F1_three-model
23,ensemble_0,"[target-iSarcasm_inter-IMDB, target-iSarcasm_i...","('target-iSarcasm_inter-IMDB_01', 'target-iSar...",iSarcasm_test,0.857143,0.494949,best-F1_five-model


In [18]:
inducers_list = top_10_f1['inducers'].to_list()

In [19]:
inducer_counts = {}

for i in inducers_list:
    for j in i:
        if j in inducer_counts:
            inducer_counts[j] += 1
        else:
            inducer_counts[j] = 1
        
inducer_counts

{'target-iSarcasm_inter-HellaSwag': 6,
 'target-iSarcasm_inter-SARC': 10,
 'target-iSarcasm_inter-XED-binary': 7,
 'target-iSarcasm_inter-XED-fine': 6,
 'target-iSarcasm_inter-CosmosQA': 6,
 'target-iSarcasm_inter-IMDB': 3}

### Ensembles Created Based on Inducer Best Epoch by Accuracy 

In [20]:
# get all ensembles created via inducer test_F1 scores
best_acc_ensembles = pd.concat([acc_TM_ensemble_results_df, acc_FM_ensemble_results_df])
len(best_acc_ensembles)

26

In [21]:
# find how many ensembles have a better test_F1 score than the best control model
better_acc_count = best_acc_ensembles['test_f1'][best_acc_ensembles['test_f1'] > control_results['test_f1'].max()].count()
better_acc_count

11

In [22]:
# find how many ensembles have a better test_accuracy score than the best control model
better_acc_count = best_acc_ensembles['test_accuracy'][best_acc_ensembles['test_accuracy'] > control_results['test_accuracy'].max()].count()
better_acc_count

26

In [23]:
# get top 10 ensembles by test_accuracy score
top_10_acc = best_acc_ensembles.nlargest(10, 'test_accuracy')
top_10_acc

,ensemble_id,inducers,inducers_spec,dataset,test_accuracy,test_f1,ensemble_type
46,ensemble_5,"[target-iSarcasm_inter-HellaSwag, target-iSarc...","('target-iSarcasm_inter-HellaSwag_01', 'target...",iSarcasm_test,0.880000,0.464968,best-acc_five-model
26,ensemble_18,"[target-iSarcasm_inter-SARC, target-iSarcasm_i...","('target-iSarcasm_inter-SARC_01', 'target-iSar...",iSarcasm_test,0.879286,0.473520,best-acc_three-model
27,ensemble_15,"[target-iSarcasm_inter-HellaSwag, target-iSarc...","('target-iSarcasm_inter-HellaSwag_01', 'target...",iSarcasm_test,0.879286,0.438538,best-acc_three-model
28,ensemble_10,"[target-iSarcasm_inter-HellaSwag, target-iSarc...","('target-iSarcasm_inter-HellaSwag_01', 'target...",iSarcasm_test,0.877857,0.501458,best-acc_three-model
29,ensemble_12,"[target-iSarcasm_inter-HellaSwag, target-iSarc...","('target-iSarcasm_inter-HellaSwag_01', 'target...",iSarcasm_test,0.877143,0.455696,best-acc_three-model
47,ensemble_1,"[target-iSarcasm_inter-IMDB, target-iSarcasm_i...","('target-iSarcasm_inter-IMDB_01', 'target-iSar...",iSarcasm_test,0.876429,0.461059,best-acc_five-model
30,ensemble_14,"[target-iSarcasm_inter-HellaSwag, target-iSarc...","('target-iSarcasm_inter-HellaSwag_01', 'target...",iSarcasm_test,0.875000,0.433657,best-acc_three-model
48,ensemble_2,"[target-iSarcasm_inter-IMDB, target-iSarcasm_i...","('target-iSarcasm_inter-IMDB_01', 'target-iSar...",iSarcasm_test,0.875000,0.440895,best-acc_five-model
31,ensemble_11,"[target-iSarcasm_inter-HellaSwag, target-iSarc...","('target-iSarcasm_inter-HellaSwag_01', 'target...",iSarcasm_test,0.874286,0.469880,best-acc_three-model
49,ensemble_4,"[target-iSarcasm_inter-IMDB, target-iSarcasm_i...","('target-iSarcasm_inter-IMDB_01', 'target-iSar...",iSarcasm_test,0.874286,0.456790,best-acc_five-model


In [24]:
inducers_list = top_10_acc['inducers'].to_list()

In [25]:
inducer_counts = {}

for i in inducers_list:
    for j in i:
        if j in inducer_counts:
            inducer_counts[j] += 1
        else:
            inducer_counts[j] = 1
        
inducer_counts

{'target-iSarcasm_inter-HellaSwag': 8,
 'target-iSarcasm_inter-SARC': 8,
 'target-iSarcasm_inter-XED-binary': 5,
 'target-iSarcasm_inter-XED-fine': 6,
 'target-iSarcasm_inter-CosmosQA': 8,
 'target-iSarcasm_inter-IMDB': 3}